In [2]:
import librosa
import numpy as np
import pandas as pd
import os

DATASET_PATH = "../data/audio/BanglaBeats" 
GENRES = ['Adhunik', 'Folk', 'Hiphop', 'Indie', 'Islamic', 'Metal', 'Pop', 'Rock']

def extract_features_to_csv(output_csv="banglabeats_3_sec.csv"):
    header = 'filename length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean perceptr_var tempo'
    for i in range(1, 21):
        header += f' mfcc{i}_mean mfcc{i}_var'
    header += ' label'
    header = header.split()

    data = []

    for genre in GENRES:
        genre_folder = os.path.join(DATASET_PATH, genre)
        print(f"Processing Genre: {genre}")
        
        for filename in os.listdir(genre_folder):
            if filename.endswith(".wav"):
                file_path = os.path.join(genre_folder, filename)
                
                # Load 3-second audio
                y, sr = librosa.load(file_path, duration=3)
                
                # Extract Features
                chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
                rms = librosa.feature.rms(y=y)
                spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
                rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
                zcr = librosa.feature.zero_crossing_rate(y)
                harmony, perceptr = librosa.effects.hpss(y)
                
                # Tempo calculation
                onset_env = librosa.onset.onset_strength(y=y, sr=sr)
                tempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr)[0]
                
                # MFCCs (1-20)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
                
                # Build Row
                res = [
                    filename, len(y), 
                    np.mean(chroma_stft), np.var(chroma_stft),
                    np.mean(rms), np.var(rms),
                    np.mean(spec_cent), np.var(spec_cent),
                    np.mean(spec_bw), np.var(spec_bw),
                    np.mean(rolloff), np.var(rolloff),
                    np.mean(zcr), np.var(zcr),
                    np.mean(harmony), np.var(harmony),
                    np.mean(perceptr), np.var(perceptr),
                    tempo
                ]
                
                for m in mfcc:
                    res.append(np.mean(m))
                    res.append(np.var(m))
                
                res.append(genre) # The folder name is our label
                data.append(res)

    # Save to DataFrame
    df = pd.DataFrame(data, columns=header)
    df.to_csv(output_csv, index=False)
    print(f"Success! {output_csv} created.")

extract_features_to_csv()

Processing Genre: Adhunik
Processing Genre: Folk
Processing Genre: Hiphop
Processing Genre: Indie
Processing Genre: Islamic
Processing Genre: Metal
Processing Genre: Pop
Processing Genre: Rock
Success! banglabeats_3_sec.csv created.
